# Usage of pKA_fitting in the nmrToolsLab package

## 1. General Information
This notebook showcases the use of the pKa_fitting tool to analyze pH titration on protein.
The NMR data have first been treated with Sparky and peak list have been extracted 

## 2. Prepare Script
Load prot_analysis.

In [1]:
import c_py_mod.prot_analysis as prot_analysis

## 3. Load peak lists
Provide all imput data (peak lists) in a dictionnary format.

In [2]:
d_experiment_ICCG_N246E_wt_His = {
    "His_CeHe_pH3.5":{"path":'./Lists', "exp":'His_pH3_5',  "peak_label": 'C-H', "pH" : 3.5, "color": "dodgerblue"},
    "His_CeHe_pH4.0":{"path":'./Lists', "exp":'His_pH4',    "peak_label": 'C-H', "pH" : 4.0, "color": "blue"},
    "His_CeHe_pH4.5":{"path":'./Lists', "exp":'His_pH4_5',  "peak_label": 'C-H', "pH" : 4.5, "color": "red"},
    "His_CeHe_pH5.0":{"path":'./Lists', "exp":'His_pH5',    "peak_label": 'C-H', "pH" : 5.0, "color": "orange"},
    "His_CeHe_pH5.5":{"path":'./Lists', "exp":'His_pH5_5',  "peak_label": 'C-H', "pH" : 5.5, "color": "green"},
    "His_CeHe_pH6.0":{"path":'./Lists', "exp":'His_pH6',    "peak_label": 'C-H', "pH" : 6.0, "color": "gray"},
    "His_CeHe_pH7.0":{"path":'./Lists', "exp":'His_pH7',    "peak_label": 'C-H', "pH" : 7.0, "color": "orchid"},
    "His_CeHe_pH8.0":{"path":'./Lists', "exp":'His_pH8',    "peak_label": 'C-H', "pH" : 8.0, "color": "black"},
    }

Load imput data 

In [3]:
pH_series_His = prot_analysis.data_consolidation(
    d_experiment_ICCG_N246E_wt_His,
    '2D',
    'pH')

Look at the consolidated data

In [4]:
print(pH_series_His.consolidated_data)

    ass   pH nucleus    shift res_type
0   291  3.5       C  140.785        H
1   291  3.5       H    8.491        H
2   291  4.0       C  140.786        H
3   291  4.0       H    8.482        H
4   291  4.5       C  140.790        H
..  ...  ...     ...      ...      ...
91  191  6.0       H    7.998        H
92  191  7.0       C  140.324        H
93  191  7.0       H    7.996        H
94  191  8.0       C  140.318        H
95  191  8.0       H    7.998        H

[96 rows x 5 columns]


## 4. Fitting examples
### 4.1. one residue, one nucleus, one pKa 

Set details for the fitting with imput as: 

* the consolidated data
* List of residue(s)
* List of nucleus (nuclei)
* model: "1" or "2" for fitting the model with 1 or 2 pKa values

In [38]:
demo_1 = prot_analysis.pKA_fitting(
    pH_series_His.consolidated_data,
    [164],
    nuclei = ['H'],
    model = 1,
    # new_params={'pKa_1':{'ini':4.5,'lb':4,'ub':5},'pKa_2':{'ini':5.5,'lb':5,'ub':6}}
    )

Show the data that are included in the fitting

In [39]:
print(demo_1.selected_data)

   ass   pH nucleus  shift res_type
0  164  3.5       H  7.495        H
1  164  4.0       H  7.495        H
2  164  4.5       H  7.532        H
3  164  5.0       H  7.604        H
4  164  5.5       H  7.712        H
5  164  6.0       H  7.826        H
6  164  7.0       H  7.944        H
7  164  8.0       H  7.967        H


Run the fitting procedure

In [40]:
demo_1.fit()

Display fitting parameters

In [52]:
print(demo_1.params)

      par par_type  ass nucleus  ini  lb  ub  model_idx  par_idx       opt  \
0     pKa      pKa                 4   2  10          0        0  5.559082   
1   d_low    shift  164       H    7   2  12          1        1  7.491268   
2  d_high    shift  164       H    7   2  12          2        2  7.962087   

     opt_sd  
0  0.000109  
1  0.000024  
2  0.000020  


Display plot. 

In [54]:
fig = demo_1.plot(res=164,fit=True)
fig.show()

### 4.2. one residue, two nuclei, one pKa 
By default if you notify two nuclei, the script will perform a global fit for this residue in which the pKa value is the common parameter.

In [43]:
demo_2 = prot_analysis.pKA_fitting(
    pH_series_His.consolidated_data,
    [164],
    nuclei = ['H','C'],
    model = 1,
    # new_params={'pKa_1':{'ini':4.5,'lb':4,'ub':5},'pKa_2':{'ini':5.5,'lb':5,'ub':6}}
    )
demo_2.fit()
fig = demo_2.plot(res=164,fit=True)
fig.show()

### 4.3. two residues, two nuclei, two pKa 
Global fit for the residues provided by the user in which both pKa (1 or 2 depending on value indicated in model) are common.
"new_params" allows to update the default parameters two disociate the 2 pKa values.  


In [48]:
demo_3 = prot_analysis.pKA_fitting(
    pH_series_His.consolidated_data,
    [164,242],
    nuclei = ['H','C'],
    model = 2,
    new_params={'pKa_1':{'ini':4.5,'lb':4,'ub':5},'pKa_2':{'ini':5.5,'lb':5,'ub':6}}
    )
demo_3.fit()
print(demo_3.params[['par','ass','opt']])
fig = demo_3.plot(res=164,fit=True)
fig.show()

         par  ass         opt
0      pKa_1         4.506976
1      pKa_2         5.600138
2      d_low  164  138.513309
3   d_high_1  164  138.580429
4   d_high_2  164  137.451800
5      d_low  164    7.962620
6   d_high_1  164    7.929154
7   d_high_2  164    7.516848
8      d_low  242  137.873522
9   d_high_1  242  135.455958
10  d_high_2  242  137.309237
11     d_low  242    7.803199
12  d_high_1  242    8.333938
13  d_high_2  242    7.857927
